In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# 1. Importamos archivo csv
df=pd.read_csv('data/attacks.csv',encoding='latin-1') #Incluir encoding='latin-1'o UTF-8/16 para que pueda abrir el archivo
print(df.shape) # ver cuantas filas/columnas tiene el archivo
df.head()

(25723, 24)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
df.columns # ver que columnas hay en el csv

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [4]:
#2. Eliminamos duplicados
df.drop_duplicates()
df.shape

(25723, 24)

In [5]:
#3.Encontrar columnas con valores nulos
df.isnull().head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True


In [6]:
#Ver total de filas nulas de cada columna
null_columns=df.isnull().sum()
null_columns[null_columns>0]

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [7]:
#Comprobar que contenido aportan las columnas "Case Number 1 and 2" diferente de la columna "Case Number"
df[["Case Number","Case Number.1","Case Number.2"]] #A primera vista parece que tiene el mismo contenido
df["Case Number.1"].equals(df["Case Number.2"].equals(df['Case Number']))#False
df["Case Number.1"].isin(df["Case Number.2"]).value_counts()# solo 19 False
df["Case Number.1"].isin(df["Case Number"]).value_counts() #Solo 23 False

# Conclusión: las columnas Case Number.1 y Case Number.2 tienen prácticamente el mismo contenido que  Case Number por lo que podemos eliminarlas


True     25700
False       23
Name: Case Number.1, dtype: int64

In [8]:
#Ademas elimino el resto de columnas que no aportan ningun valor al analisis: "pdf","href formula","href","Unnamed: 22","Unnamed: 23","original order","Investigator or Source"

df_clean = df.drop(columns=["Case Number.1","Case Number.2","pdf","href formula","href","Unnamed: 22","Unnamed: 23","original order","Investigator or Source"])
df_clean.shape

(25723, 15)

In [9]:
df_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species '],
      dtype='object')

In [10]:
df_clean.isnull().sum()

Case Number    17021
Date           19421
Year           19423
Type           19425
Country        19471
Area           19876
Location       19961
Activity       19965
Name           19631
Sex            19986
Age            22252
Injury         19449
Fatal (Y/N)    19960
Time           22775
Species        22259
dtype: int64

In [11]:
#Drop the rows where all elements are missing -> al haber tantas rows vacias elimino las filas que tengan todas las columnas nulas
df_clean=df_clean.dropna(how='all')
df_clean.shape

(8703, 15)

In [12]:
df_clean.isnull().sum()

Case Number       1
Date           2401
Year           2403
Type           2405
Country        2451
Area           2856
Location       2941
Activity       2945
Name           2611
Sex            2966
Age            5232
Injury         2429
Fatal (Y/N)    2940
Time           5755
Species        5239
dtype: int64

In [13]:
# Limpio columna "Case Number"
df_clean["Case Number"].value_counts() # Parece que el contenido de la columna es igual a la fecha= Columna Date/Year
df_clean[df_clean["Case Number"].isnull()][["Year","Date"]] # Compruebo la columna nula con columnas relacionadas

,Year,Date
5488,1905.0,Reported 06-Sep-1905


In [14]:
df_clean["Case Number"]= df_clean["Case Number"].fillna("1905.09.06") # Relleno info nula con info de columnas relacionadas

In [15]:
df_clean.isnull().sum()

Case Number       0
Date           2401
Year           2403
Type           2405
Country        2451
Area           2856
Location       2941
Activity       2945
Name           2611
Sex            2966
Age            5232
Injury         2429
Fatal (Y/N)    2940
Time           5755
Species        5239
dtype: int64

In [16]:
# Elimino todas las filas que tengan por lo menos 12 columnas nulas (hay 15 en total)
df_clean=df_clean.dropna(thresh=12)
df_clean.shape

(5343, 15)

In [17]:
df_clean.isnull().sum()

Case Number       0
Date              0
Year              0
Type              1
Country           2
Area            104
Location        144
Activity        155
Name             24
Sex             217
Age            1936
Injury            3
Fatal (Y/N)     365
Time           2430
Species        2144
dtype: int64

In [18]:
# Limpio columna "Type"
df_clean["Type"].value_counts()
df_clean[df_clean["Type"].isnull()][["Activity","Injury","Fatal (Y/N)"]]

,Activity,Injury,Fatal (Y/N)
85,Fishing,Injuries to hands and legs,N


In [19]:
#Añado fila vacia y adapto contenido de la columna para que quede categorizado
df_clean["Type"]= df_clean["Type"].fillna("Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinginging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingingomg","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Questionable","Invalid")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boating","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boat","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingingomg","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingomg","Boating")
df_clean["Type"].value_counts()

Unprovoked      4076
Provoked         520
Invalid          383
Boating          254
Sea Disaster     110
Name: Type, dtype: int64

In [20]:
df_clean.isnull().sum()

Case Number       0
Date              0
Year              0
Type              0
Country           2
Area            104
Location        144
Activity        155
Name             24
Sex             217
Age            1936
Injury            3
Fatal (Y/N)     365
Time           2430
Species        2144
dtype: int64

In [21]:
#Limpio columna "Country"
df_clean["Country"].value_counts()
df_clean[df_clean["Country"].isnull()][["Area","Location","Activity"]]

,Area,Location,Activity
4231,Near the Andaman & Nicobar Islands,NaN,Climbing back on ship
4498,Caribbean Sea,Between Cuba & Costa Rica,"Sea Disaster, sinking of the motorship Wingate"


In [22]:
df_clean.loc[df_clean["Country"].isnull(),"Country"] = ["INDIA","PANAMA"] #Relleno filas nulas
df_clean["Country"]=df_clean["Country"].apply(lambda x: x.lower())
df_clean["Country"]=df_clean["Country"].str.replace('(\w+)', lambda x: x.group(0).capitalize())
df_clean["Country"]

0                      Usa
1                      Usa
2                      Usa
3                Australia
4                   Mexico
               ...        
6290          South Africa
6296             Australia
6297             Australia
6299                   Usa
6301    Ceylon (Sri Lanka)
Name: Country, Length: 5343, dtype: object

In [23]:
#Relleno filas nulas del resto de columnas para que quede a 0
df_clean["Area"]= df_clean["Area"].fillna("Unknown")
df_clean["Location"]= df_clean["Location"].fillna("Unknown")
df_clean["Activity"]= df_clean["Activity"].fillna("Unknown")
df_clean["Name"]= df_clean["Name"].fillna("Noname")
df_clean["Sex "]= df_clean["Sex "].fillna("Unknown")
df_clean["Age"]= df_clean["Age"].fillna("0")
df_clean["Injury"]= df_clean["Injury"].fillna("Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].fillna("Unknown")
df_clean["Time"]= df_clean["Time"].fillna("Notime")
df_clean["Species "]= df_clean["Species "].fillna("Unknown")

# ALL DATA IS CLEANED
df_clean.isnull().sum()[df_clean.isnull().sum() > 0]

Series([], dtype: int64)

In [24]:
#Verifico tipo de datos en cada columna 
df_clean.dtypes #Todos son object menos Year que es float64
df_clean["Year"]=df_clean["Year"].astype("object")
df_clean["Year"].dtype

dtype('O')

In [25]:
#Corrijo nombres de algunas columnas con espacios
df_clean=df_clean.rename(columns={"Sex ":"Sex"}) #Quito el espacio del nombre de la columna "Sex"
df_clean=df_clean.rename(columns={"Species ":"Species"})

In [26]:
#Limpio contenido columna Activity 
df_clean.loc[df_clean["Activity"].str.contains("wim"),"Activity"]="Swimming"
df_clean.loc[df_clean["Activity"].str.contains("urf"),"Activity"]="Surfing"
df_clean.loc[df_clean["Activity"].str.contains("ish"),"Activity"]="Fishing"
df_clean["Activity"]= df_clean["Activity"].str.replace("Spearfishing","Fishing")
df_clean["Activity"].value_counts()

Surfing                                   1225
Fishing                                   1076
Swimming                                  1027
Unknown                                    158
Wading                                     146
                                          ... 
Playing with a frisbee in the shallows       1
Fell oveboard from steamer Wanderer          1
Landing hooked shark in boat                 1
Dry shelling                                 1
Washing horses                               1
Name: Activity, Length: 721, dtype: int64

In [27]:
#Limpio contenido columnas relacionadas Case Number, Year, Date

df_clean["Date"].value_counts() # Parece que hay diferentes formatos de fecha
df_clean["Case Number"].value_counts() # Podemos utilzar estos formatos de fecha que estan emejor que columna Date  

1990.05.10      2
1980.07.00      2
2013.10.05      2
2012.09.02.b    2
2006.09.02      2
               ..
1890.00.00.e    1
1995.10.11      1
2012.04.11      1
2008.07.09      1
2015.08.18.a    1
Name: Case Number, Length: 5330, dtype: int64

In [28]:
set(df_clean["Date"])
set(df_clean["Case Number"])

{'2015.08.18.a',
 '2009.02.22',
 '1989.01.20.a',
 '1951.03.26',
 '2008.04.15',
 '1998.12.22',
 '1999.03.18.R',
 '1969.04.22',
 '2010.04.13',
 '1959.04.27',
 '1948.12.10',
 '2015.05.20',
 '1931.08.25',
 '1962.06.17',
 '1997.08.09',
 '1964.07.27',
 '1894.12.11',
 '2013.08.26',
 '1909.08.13',
 '1980.04.25',
 '1922.04.26',
 '1972.12.31',
 'ND.0003',
 '1839.00.00.a',
 '1966.08.27.b',
 '2008.07.09',
 '1950.02.11',
 '1905.01.01',
 '1767.00.00',
 '2004.01.07',
 '1985.09.28',
 '1995.09.11',
 '2012.03.01',
 '2003.04.20.b',
 '2009.01.25',
 '1977.01.00',
 '1999.09.24',
 '1931.06.13',
 '2009.04.21',
 '1995.06.16',
 '2010.02.01',
 '2016.07.15.b',
 '2017.11.18',
 '2011.06.06.R',
 '1954.01.15',
 'ND.0054',
 '2003.05.07',
 '1960.10.24',
 '1993.08.19',
 '1889.07.21',
 '2003.06.08',
 '1853.09.28',
 '1998.07.06.b',
 '2014.08.24',
 '2003.12.13.a',
 '1999.09.29',
 '1988.03.31',
 '1997.08.27',
 '1988.06.19',
 '1999.10.20',
 '1964.01.23',
 '1963.09.29',
 '1983.07.05',
 '2017.03.27',
 '1988.01.06',
 '1984.11.1

In [29]:
#Al estar en mejor formato la columna "Case Number" vamos a limpiar esta para utilizar su contenido
df_clean["Case Number"]=df_clean["Case Number"].map(lambda x: str(x)[0:10]) # quito todos los elementos fuera de la fecha
df_clean.loc[df_clean["Case Number"].str.startswith("ND"),"Case Number"]="0000.00.00" 
set(df_clean["Case Number"])

{'1966.12.27',
 '2009.02.22',
 '1993.06.05',
 '1943.05.14',
 '1968.11.02',
 '1954.07.15',
 '2016.06.05',
 '1922.06.17',
 '1895.11.21',
 '2007.02.03',
 '2010.07.23',
 '2018.04.30',
 '1962.06.10',
 '2008.04.15',
 '2006.05.24',
 '1951.03.26',
 '2001.10.07',
 '2007.03.05',
 '1998.12.22',
 '2004.02.11',
 '1995.10.11',
 '2016.07.08',
 '2009.04.19',
 '1984.09.23',
 '1969.04.22',
 '1899.08.23',
 '1986.08.10',
 '1961.07.29',
 '2010.04.13',
 '1942.03.08',
 '2007.08.12',
 '1999.08.16',
 '1959.05.03',
 '1935.03.25',
 '1901.09.23',
 '1959.04.27',
 '2011.04.16',
 '2009.09.12',
 '1961.04.25',
 '1956.01.16',
 '1900.09.05',
 '2007.07.17',
 '1997.08.11',
 '1964.01.21',
 '2005.06.05',
 '2004.10.01',
 '2015.05.09',
 '1960.04.24',
 '2009.07.24',
 '2016.02.12',
 '1968.03.00',
 '1949.07.16',
 '1997.09.06',
 '2015.08.20',
 '2015.05.20',
 '2004.11.15',
 '1957.12.30',
 '1948.12.10',
 '1931.08.25',
 '2000.03.24',
 '1962.06.17',
 '1828.09.28',
 '1998.12.24',
 '2015.01.06',
 '2011.05.07',
 '2017.07.23',
 '1997.08.

In [30]:
# Creo nuevas columnas para diferenciar meses y años de cada fecha en base a columna "Case Number"
df_clean['Day']=[d[8:10] for d in df_clean['Case Number']]
df_clean['Month']=[d[5:7] for d in df_clean['Case Number']]
df_clean["Year2"]=[d[0:4] for d in df_clean['Case Number']]

In [31]:
df_clean[["Year2","Year"]].head() # Comparo para ver si Year2 y Year tienen mismo contenido

,Year2,Year
0,2018,2018
1,2018,2018
2,2018,2018
3,2018,2018
4,2018,2018


In [32]:
df_clean["Year2"]=df_clean["Year2"].astype("int") # Convierto columna a int para poder crear columna Decade

In [33]:
# Creo columna Decade para poder categorizar los años
df_clean["Decade"] = pd.cut(df_clean.Year2,
                                bins=[1968,1973,1978,1983,1988,1993,1998,2003,2008,2013,2018], 
                                labels=range(1970,2020,5))

df_clean["Decade"]= df_clean["Decade"].fillna(1970) # Relleno fila nula (añoss anteriores a 1908 las incluyo en la categoria 1910)

df_clean["Year2"]=df_clean["Year2"].astype("object") # vuelvo a convertir la columna a object

df_clean["Decade"].value_counts()


1970    2300
2010     565
2015     557
2005     506
2000     407
1995     281
1990     228
1985     178
1980     172
1975     149
Name: Decade, dtype: int64

In [34]:
# Creo nuevo columna de "Seasonality" para categorizar el tipo de season por mes
seasons_month = {"Winter": ["12", "01", "02"],
             "Spring": ["03", "04", "05"],
             "Summer": ["06", "07", "08"],
             "Autumn": ["09", "10", "11"],
             "Unknown": ["00", "94"]}
Seasonality_name=[]
for x in df_clean["Month"]:
    if x in seasons_month["Winter"]:
        Seasonality_name.append("Winter")
    elif x in seasons_month["Spring"]:
        Seasonality_name.append("Spring")
    elif x in seasons_month["Summer"]:
        Seasonality_name.append("Summer")
    elif x in seasons_month["Autumn"]:
        Seasonality_name.append("Autumn")
    elif x in seasons_month["Unknown"]:
        Seasonality_name.append("Unknown")

df_clean["Seasonality"]= Seasonality_name

In [35]:
#Limpio columna "Fatal (Y/N) para quede categorizada
df_clean["Fatal (Y/N)"].value_counts()
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace(" N","N")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("2017","Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("M","Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("UNKNOWN","Unknown")

In [36]:
df_clean["Fatal (Y/N)"].value_counts()

N          3899
Y          1054
Unknown     390
Name: Fatal (Y/N), dtype: int64

In [37]:
#Limpio columna Injury

df_clean["Injury"]= df_clean["Injury"].str.replace("FATAL","Fatal")
df_clean["Injury"]= df_clean["Injury"].str.replace("Foot bitten","Injury")
df_clean["Injury"]= df_clean["Injury"].str.replace("Leg bitten","Injury")
df_clean.loc[df_clean["Injury"].str.contains("atal"),"Injury"]="Fatal"
df_clean.loc[df_clean["Injury"].str.contains("No injury"),"Injury"]="No injury"
df_clean.loc[df_clean["Injury"].str.contains("Foot"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("arm"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("leg"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("hand"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("bit"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("njur"),"Injury"]="Injury"

In [38]:
#Limpio columna "Age"
df_clean["Age"].value_counts()
#Paso todo lo que no es digito a digito
df_clean["Age"]= df_clean["Age"].str.replace("teen","15")
df_clean["Age"]= df_clean["Age"].str.replace("Teen","15")
df_clean["Age"]= df_clean["Age"].str.replace("mid-30s","30")
df_clean["Age"]= df_clean["Age"].str.replace("mid-20s","20")
df_clean["Age"]= df_clean["Age"].str.replace("adult","45")
df_clean["Age"]= df_clean["Age"].str.replace("(adult)","45")
df_clean["Age"]= df_clean["Age"].str.replace("young","22")
df_clean["Age"]= df_clean["Age"].str.replace("(young)","22")
df_clean["Age"]= df_clean["Age"].str.replace("Adult","45")
df_clean["Age"]= df_clean["Age"].str.replace(">50","50")
df_clean["Age"]= df_clean["Age"].str.replace("Elderly","70")

In [39]:
df_clean["Age"].value_counts() 

0           1936
15           151
17           151
18           149
19           140
            ... 
2½             1
1              1
25 to 35       1
 30            1
A.M.           1
Name: Age, Length: 149, dtype: int64

In [40]:
df_clean[df_clean["Age"].str.match('^(?![0,1,2,3,4,5,6,7,8,9])')]
df_clean["Age"]=df_clean["Age"].str.extract('(\d+)', expand=False)
df_clean["Age"]= df_clean["Age"].fillna("0")# al generarse nuevas lineas nulas, las reemplazo por 0
df_clean.isnull().sum()

Case Number    0
Date           0
Year           0
Type           0
Country        0
Area           0
Location       0
Activity       0
Name           0
Sex            0
Age            0
Injury         0
Fatal (Y/N)    0
Time           0
Species        0
Day            0
Month          0
Year2          0
Decade         0
Seasonality    0
dtype: int64

In [41]:
df_clean["Age"]=df_clean["Age"].astype("int") # convierto Age en int para crear nueva categoria "Age Category"
Age_category=[]

for x in df_clean["Age"]:
    if x >= 65:
        Age_category.append("Elderly")
    elif x >= 35:
        Age_category.append("Adults")
    elif x >= 20:
        Age_category.append("Young")
    elif x >= 10:
        Age_category.append("Teens")
    elif 0 > x < 10:
        Age_category.append("Children")
    else:
        Age_category.append("Unknown")

        
df_clean["Age Category"]=Age_category     

In [42]:
#Categorizamos columna Sex
df_clean["Sex"].value_counts()
df_clean["Sex"]= df_clean["Sex"].str.replace(".","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("lli","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("N","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("M ","M")
df_clean["Sex"].value_counts()

M          4536
F           587
Unknown     220
Name: Sex, dtype: int64

In [43]:
df_clean= df_clean.drop_duplicates()

In [44]:
df_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Day', 'Month', 'Year2', 'Decade', 'Seasonality',
       'Age Category'],
      dtype='object')

In [45]:
#Ordeno orden columnas
column_order = ["Case Number","Date","Year","Time","Day","Month","Year2","Decade","Seasonality","Country","Area","Location","Type","Activity","Injury","Fatal (Y/N)","Species","Name","Sex","Age","Age Category"]
df_clean=df_clean[column_order]

In [46]:
df_analysis = df_clean[["Case Number","Date","Year","Time","Day","Month","Year2","Decade","Seasonality","Country","Area","Location","Type","Activity","Injury","Fatal (Y/N)","Species","Name","Sex","Age","Age Category"]]

In [47]:
df_analysis.to_csv("data/attacks_limpio.csv")